In [1]:
from datasets import *
from train import *
from torchvision import datasets, transforms, models

training

In [34]:
model = models.mobilenet_v3_small(weights='DEFAULT').to('cuda')
with torch.no_grad():
    sub_w1 = model.classifier[0].weight[:,:]
    sub_b1 = model.classifier[0].bias[:]
    new_layer1 = torch.nn.Linear(sub_w1.size()[1],1280)

    sub_w2 = model.classifier[3].weight[:,:]
    sub_b2 = model.classifier[3].bias[:]
    new_layer2 = torch.nn.Linear(1280,1000)

    # copy pretrained part of the weight matrix
    new_layer1.weight[:sub_w1.size()[0],:] = sub_w1
    new_layer1.bias[:sub_b1.size()[0]] = sub_b1

    new_layer2.weight[:,:sub_w2.size()[1]] = sub_w2
    new_layer2.bias[:sub_b2.size()[0]] = sub_b2

    model.classifier[0] = new_layer1
    model.classifier[3] = new_layer2
    print(model.classifier)
    # model.classifier[0] = torch.nn.Linear(576,1280)
    # model.classifier[3] = torch.nn.Linear(1280,1000)
# model = models.mobilenet_v3_large()
# model.load_state_dict(torch.load("best_batch_i162153.pth")['model_state_dict'])
# train_loader, val_loader,_ = load_tiny_imagenet(64,1234)
train_loader, val_loader = load_imagenet(64,12345,resize=128)
train(model,train_loader,val_loader,'cuda',lr=0.004)

Sequential(
  (0): Linear(in_features=576, out_features=1280, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1280, out_features=1000, bias=True)
)
Train Epoch: 0 [0/1153050 (0%)] train loss: 3.761, lr: 0.00400000
Train Epoch: 0 [51200/1153050 (4%)] train loss: 1.661, lr: 0.00396000
Train Epoch: 0 [102400/1153050 (9%)] train loss: 2.307, lr: 0.00392040
Train Epoch: 0 [153600/1153050 (13%)] train loss: 2.407, lr: 0.00388120
Train Epoch: 0 [204800/1153050 (18%)] train loss: 1.932, lr: 0.00384238
Train Epoch: 0 [256000/1153050 (22%)] train loss: 2.057, lr: 0.00380396
Train Epoch: 0 [307200/1153050 (27%)] train loss: 1.654, lr: 0.00376592
Train Epoch: 0 [358400/1153050 (31%)] train loss: 2.204, lr: 0.00372826
Train Epoch: 0 [409600/1153050 (36%)] train loss: 1.713, lr: 0.00369098
Train Epoch: 0 [460800/1153050 (40%)] train loss: 1.780, lr: 0.00365407
Train Epoch: 0 [512000/1153050 (44%)] train loss: 1.801, lr: 0.00361753
Train Epoch: 0 [563200/11

100%|██████████| 2002/2002 [02:31<00:00, 13.26it/s]


Train Epoch: 0 [468416/1153050 (100%)] train loss: 2.143, val loss: 1.887, val acc: 0.555, top5: 0.791, lr: 0.00317446
==================== best validation accuracy ====================
epoch: 0, val accuracy: 0.5552502790418133
Train Epoch: 1 [0/1153050 (0%)] train loss: 2.061, lr: 0.00317446
Train Epoch: 1 [51200/1153050 (4%)] train loss: 1.490, lr: 0.00157136
Train Epoch: 1 [102400/1153050 (9%)] train loss: 1.473, lr: 0.00155564
Train Epoch: 1 [153600/1153050 (13%)] train loss: 1.279, lr: 0.00154009
Train Epoch: 1 [204800/1153050 (18%)] train loss: 1.567, lr: 0.00152469
Train Epoch: 1 [256000/1153050 (22%)] train loss: 1.320, lr: 0.00150944
Train Epoch: 1 [307200/1153050 (27%)] train loss: 1.413, lr: 0.00149434
Train Epoch: 1 [358400/1153050 (31%)] train loss: 1.461, lr: 0.00147940
Train Epoch: 1 [409600/1153050 (36%)] train loss: 2.039, lr: 0.00146461
Train Epoch: 1 [460800/1153050 (40%)] train loss: 1.464, lr: 0.00144996
Train Epoch: 1 [512000/1153050 (44%)] train loss: 1.625, lr:

100%|██████████| 2002/2002 [02:30<00:00, 13.32it/s]


Train Epoch: 1 [468416/1153050 (100%)] train loss: 1.447, val loss: 1.479, val acc: 0.633, top5: 0.853, lr: 0.00125965
==================== best validation accuracy ====================
epoch: 1, val accuracy: 0.63317124191169
Train Epoch: 2 [0/1153050 (0%)] train loss: 1.316, lr: 0.00125965
Train Epoch: 2 [51200/1153050 (4%)] train loss: 1.186, lr: 0.00062353
Train Epoch: 2 [102400/1153050 (9%)] train loss: 1.017, lr: 0.00061729
Train Epoch: 2 [153600/1153050 (13%)] train loss: 0.915, lr: 0.00061112
Train Epoch: 2 [204800/1153050 (18%)] train loss: 1.367, lr: 0.00060501
Train Epoch: 2 [256000/1153050 (22%)] train loss: 1.346, lr: 0.00059896
Train Epoch: 2 [307200/1153050 (27%)] train loss: 0.936, lr: 0.00059297
Train Epoch: 2 [358400/1153050 (31%)] train loss: 1.351, lr: 0.00058704
Train Epoch: 2 [409600/1153050 (36%)] train loss: 1.056, lr: 0.00058117
Train Epoch: 2 [460800/1153050 (40%)] train loss: 1.366, lr: 0.00057535
Train Epoch: 2 [512000/1153050 (44%)] train loss: 1.634, lr: 0

...

Train Epoch: 8 [0/1153050 (0%)] train loss: 0.647, lr: 0.00000492
Train Epoch: 8 [51200/1153050 (4%)] train loss: 0.712, lr: 0.00000243
Train Epoch: 8 [102400/1153050 (9%)] train loss: 1.434, lr: 0.00000241
Train Epoch: 8 [153600/1153050 (13%)] train loss: 1.191, lr: 0.00000239
Train Epoch: 8 [204800/1153050 (18%)] train loss: 1.021, lr: 0.00000236
Train Epoch: 8 [256000/1153050 (22%)] train loss: 0.822, lr: 0.00000234
Train Epoch: 8 [307200/1153050 (27%)] train loss: 1.070, lr: 0.00000231
Train Epoch: 8 [358400/1153050 (31%)] train loss: 0.972, lr: 0.00000229
Train Epoch: 8 [409600/1153050 (36%)] train loss: 0.930, lr: 0.00000227
Train Epoch: 8 [460800/1153050 (40%)] train loss: 1.317, lr: 0.00000225
Train Epoch: 8 [512000/1153050 (44%)] train loss: 0.783, lr: 0.00000222
Train Epoch: 8 [563200/1153050 (49%)] train loss: 1.002, lr: 0.00000220
Train Epoch: 8 [614400/1153050 (53%)] train loss: 0.905, lr: 0.00000218
Train Epoch: 8 [665600/1153050 (58%)] train loss: 1.187, lr: 0.00000216
T

100%|██████████| 2002/2002 [02:34<00:00, 12.99it/s]

Train Epoch: 8 [468416/1153050 (100%)] train loss: 1.359, val loss: 1.348, val acc: 0.663, top5: 0.871, lr: 0.00000195
==================== best validation accuracy ====================
epoch: 8, val accuracy: 0.6629331002130865


Train Epoch: 9 [0/1153050 (0%)] train loss: 1.316, lr: 0.00000195


KeyboardInterrupt: 

testing

In [35]:
test_loader = load_imagenet(128,1234,False)
test_loader128 = load_imagenet(128,1234,False,resize=128)

In [36]:
model = models.mobilenet_v3_large(weights='DEFAULT').to('cuda')
model64 = models.mobilenet_v3_small()
model64.classifier[0] = torch.nn.Linear(576,1280)
model64.classifier[3] = torch.nn.Linear(1280,1000)
model64.load_state_dict(torch.load("best_batch_i162153.pth")['model_state_dict'])
model64 = model64.to('cuda')
model.eval()
model64.eval()

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [37]:
acc64_clean = validate(model64,test_loader128,'cuda')

100%|██████████| 391/391 [04:11<00:00,  1.56it/s]


In [38]:
acc64_clean

(0.57538, 1.8619217997621698, 0.80626)

In [9]:
acc_clean = validate(model,test_loader,'cuda')
acc64_clean = validate(model64,test_loader128,'cuda')

100%|██████████| 2002/2002 [02:33<00:00, 13.06it/s]


In [10]:
acc64_clean

(0.41572156700515933, 2.7379780524855963, 0.6734781488795398)

In [8]:
print("Mobilenet V3 Large Clean (224x224)\n Top 1: {:2.2f}%\t Top 5: {:2.2f}%".format(acc_clean[0]*100,acc_clean[2]*100))
print("Mobilenet V3 Large Clean (64x64)\n Top 1: {:2.2f}%\t Top 5: {:2.2f}%".format(acc64_clean[0]*100,acc64_clean[2]*100))

Mobilenet V3 Large Clean (224x224)
 Top 1: 75.31%	 Top 5: 92.63%
Mobilenet V3 Large Clean (64x64)
 Top 1: 38.34%	 Top 5: 63.53%


In [27]:
from thop import profile
model = model = models.mobilenet_v3_large()
input = torch.randn(1, 3, 224, 224)
macs, params = profile(model, inputs=(input, ))
print("224x224:",macs/1e6,params)

model = models.mobilenet_v3_small(weights='DEFAULT')
model.classifier[0] = torch.nn.Linear(576,1280)
model.classifier[3] = torch.nn.Linear(1280,1000)
input = torch.randn(1, 3, 128, 128)
macs, params = profile(model, inputs=(input, ))
print("64x64:",macs/1e6,params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
224x224: 22.863256 5483032.0
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container